In [2]:
################ ---------------ETL ------------------

# Dependencies and Setup

%matplotlib inline
# Dependencies
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
from pprint import pprint
import scipy.stats as stats

In [3]:
PM_data = "PM_ML.csv"
PM_data = pd.read_csv(PM_data)
PM_data_rename = PM_data.rename(columns={"Radar_Availability(%)": "availability",
                                                        "Radar_CS(Call_Completion)": "cs_call_completion",
                                                        "Radar_PS(Call_Completion)": "ps_call_completion",
                                                        "Radar_Throughput_R17(kbit/s)": "throughput_kbps",
                                                        "Radar_RTWP": "rtwp",
                                                        "Radar_Cell Erlang Equivalents": "traffic",
                                                        "6.PS_TRAFFIC VOLUME_DL_HSDPA(MB)": "traffic_mb",
                                                        "VS.TP.UE.Mean": "propagation_mts",
                                                        "VS.Periodic.MrRpt.EcNo.MEAN(dB)": "quality_ecno",
                                                        "GoodRrcAccessEcnoSamples12dB(%)": "goodquality",
                                                        "VS.HSDPA.UE.Mean.Cell": "users_data",
                                                        "VS.CellDCHUEs": "users_total",
                                                        "Date":"dates",
                                                        "RNC":"rnc",
                                                        "NodeB Name":"nodebname",
                                                        "VS.Periodic.MrRpt.RSCP.MEAN": "coverage_rscp",
                                        "Cell ID":"cellid","Cell Name":"cellname"})
PM_data_rename.index.names = ['id']
KPM_data_rename_clean=PM_data_rename.dropna(axis = 0, how ='any')

KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.cs_call_completion != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.cs_call_completion != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.ps_call_completion != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.ps_call_completion!= "NIL"]
KPM_data_rename_clean= KPM_data_rename_clean[KPM_data_rename_clean.throughput_kbps  != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.throughput_kbps != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.rtwp   != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.rtwp  != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.availability  != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.availability  != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.traffic   != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.traffic   != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.traffic_mb  != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.traffic_mb  != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.propagation_mts   != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.propagation_mts   != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.quality_ecno   != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.quality_ecno   != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.goodquality   != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.goodquality   != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.users_data    != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.users_data    != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.users_total   != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.users_total   != "NIL"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.coverage_rscp     != "/0"]
KPM_data_rename_clean = KPM_data_rename_clean[KPM_data_rename_clean.coverage_rscp    != "NIL"]

KPM_data_rename_clean.columns

Index(['dates', 'rnc', 'cellid', 'cellname', 'nodebname', 'availability',
       'cs_call_completion', 'ps_call_completion', 'throughput_kbps', 'rtwp',
       'traffic', 'traffic_mb', 'propagation_mts', 'quality_ecno',
       'goodquality', 'users_data', 'users_total', 'coverage_rscp'],
      dtype='object')

In [4]:
Cell_Level = "Cell_Level_Report_Week44.csv"
Cell_Level_data = pd.read_csv(Cell_Level, encoding = "ISO-8859-1")
Cell_Radar=Cell_Level_data[['[SectorName]','AZIMUTH','FREQUENCY_BAND','[PrimaryCpichPower]','[MaxPower]','[PD_85]']].copy()
Cell_Radar= Cell_Radar.rename(columns={"[SectorName]":"cellname","AZIMUTH":"azimuth","FREQUENCY_BAND":"banda",
                                      "[PrimaryCpichPower]":"cpich","[MaxPower]":"maxpower","[PD_85]":"tp_85%"})
Cell_Radar.columns

C:\Programas\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['cellname', 'azimuth', 'banda', 'cpich', 'maxpower', 'tp_85%'], dtype='object')

In [5]:
Radar = "Radar_w44.csv"
Radar_data = pd.read_csv(Radar) 
Radar=Radar_data[['CellName','Traffic_Load']].copy()
Radar= Radar.rename(columns={"CellName":"cellname","Traffic_Load":"traffic_load"})
Radar.columns

Index(['cellname', 'traffic_load'], dtype='object')

In [6]:
OVERSHOOTER_R5= "OVERSHOOTER UMTS_W44.csv"
OVERSHOOTER_R5 = pd.read_csv(OVERSHOOTER_R5, encoding = "ISO-8859-1")
OVERSHOOTER_R5= OVERSHOOTER_R5.rename(columns={"Cell Name":"cellname","Distance":"neigh_dist" })
OVERSHOOTER_R5=OVERSHOOTER_R5[['cellname','COI','neigh_dist','Morphology','Overshooting']].copy()
OVERSHOOTER_R5= OVERSHOOTER_R5.rename(columns={"COI":"coi","Morphology":"morphology","Overshooting":"overshooting" })
OVERSHOOTER_R5.columns

Index(['cellname', 'coi', 'neigh_dist', 'morphology', 'overshooting'], dtype='object')

In [7]:
CM_data = "EPT_ALL_W44.csv"
CMData_Nodes_df= pd.read_csv(CM_data, encoding = "ISO-8859-1") 
New_CMData_Nodes_df= CMData_Nodes_df[['CELL','RNC',
                                    'Site','SITE NAME',
                                    'Azimuth','Height',
                                    'Latitude','Longitude','Mech Downtilt','Horizontal Beam Width',
                                     'Vertical Beam Width']].copy()
New_CMData_Nodes_df = New_CMData_Nodes_df.iloc[1:]
New_CMData_Nodes_df  = New_CMData_Nodes_df .rename(columns={"CELL": "cellname",
                                                        "RNC": "rnc",
                                                        "Site": "site",
                                                        "SITE NAME": "site_name",
                                                        "Azimuth": "azimuth",
                                                        "Height": "height",
                                                        "Latitude": "latitude",
                                                        "Longitude": "longitude",
                                                           "Mech Downtilt":"mechtilt",
                                                          "Horizontal Beam Width":"hbwidth",
                                                           "Vertical Beam Width":"vbwidth"})
New_CMData_Nodes_df.index.names = ['id']
New_CMData_Nodes_df.columns

Index(['cellname', 'rnc', 'site', 'site_name', 'azimuth', 'height', 'latitude',
       'longitude', 'mechtilt', 'hbwidth', 'vbwidth'],
      dtype='object')

In [24]:
Tilt_data = "tilt_v4.csv"
Tilt_data_df= pd.read_csv(Tilt_data, encoding = "ISO-8859-1") 
Tilt_final= Tilt_data_df.rename(columns={"Cell Name":"cellname","Actual Tilt(0.1degree)":"tilt" })
Tilt_final.head()

,cellname,tilt
0,JLU2094,15
1,JLX2094,15
2,JLW2094,15
3,JLZ2094,15
4,JLU2558,15


In [25]:
ML_model= pd.merge(KPM_data_rename_clean,Cell_Radar, on='cellname', how='left')
ML_model= pd.merge(ML_model,Radar, on='cellname', how='left')
ML_model= pd.merge(ML_model,OVERSHOOTER_R5, on='cellname', how='left')
ML_model= pd.merge(ML_model,New_CMData_Nodes_df, on='cellname', how='left')
ML_model= pd.merge(ML_model,Tilt_final, on='cellname', how='left')
ML_data=ML_model[['cs_call_completion','ps_call_completion','throughput_kbps','rtwp','traffic',
                                    'quality_ecno','goodquality',
                 'users_data','users_total','coverage_rscp','banda','cpich',
                 'maxpower','traffic_load','tp_85%','morphology','height','mechtilt',
                 'hbwidth','vbwidth','tilt']].copy()
ML_data = ML_data.dropna(axis='columns', how='all')
ML_data = ML_data.dropna()
convert_dict = {'cs_call_completion': float, 
                'ps_call_completion': float,
                'throughput_kbps':float,
                'rtwp':float,
                'traffic':float,
                'quality_ecno':float,
                'goodquality':float,
                'users_data':float,
                'users_total':float,
                'coverage_rscp':float            
                          }
ML_data = ML_data.astype(convert_dict)
ML_data.to_csv (r'C:\Vicvaporub\Bootcamp\Project Final\Resumen\JupyterNotebook\ML_database_v2.csv', 
                          index = None, header=True)

In [26]:
ML_data_denseurban = ML_data.loc[ML_data.morphology == "DU"]
ML_data_urban = ML_data.loc[ML_data.morphology == "UR"]
ML_data_rural = ML_data.loc[ML_data.morphology == "SU/RU"]
ML_data_denseurban=ML_data_denseurban.drop(columns=["morphology"])
ML_data_urban=ML_data_urban.drop(columns=["morphology"])
ML_data_rural=ML_data_rural.drop(columns=["morphology"])
y_denseurban = ML_data_denseurban["tilt"]
X_denseurban = ML_data_denseurban.drop(columns=["tilt"])
y_urban = ML_data_urban["tilt"]
X_urban = ML_data_urban.drop(columns=["tilt"])
y_rural = ML_data_rural["tilt"]
X_rural = ML_data_rural.drop(columns=["tilt"])

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_denseurban, y_denseurban, random_state=1, stratify=y_denseurban)
X_train_urban, X_test_urban, y_train_urban, y_test_urban = train_test_split(X_urban, y_urban, random_state=1, stratify=y_urban)
X_train_rural, X_test_rural, y_train_rural, y_test_rural = train_test_split(X_rural, y_rural, random_state=1, stratify=y_rural)

In [28]:
y_test.to_csv (r'C:\Vicvaporub\Bootcamp\Project Final\Resumen\JupyterNotebook\y_test.csv', 
                          index = None, header=True)
y_test_urban.to_csv (r'C:\Vicvaporub\Bootcamp\Project Final\Resumen\JupyterNotebook\y_test_urban.csv', 
                          index = None, header=True)
y_test_rural.to_csv (r'C:\Vicvaporub\Bootcamp\Project Final\Resumen\JupyterNotebook\y_test_rural.csv', 
                          index = None, header=True)

In [29]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_scaler_urban = MinMaxScaler().fit(X_train_urban)
X_train_scaled_urban = X_scaler.transform(X_train_urban)
X_test_scaled_urban = X_scaler.transform(X_test_urban)
X_scaler_rural = MinMaxScaler().fit(X_train_rural)
X_train_scaled_rural = X_scaler.transform(X_train_rural)
X_test_scaled_rural = X_scaler.transform(X_test_rural)

In [30]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

In [31]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
label_encoder = LabelEncoder()
label_encoder.fit(y_train_urban)
encoded_y_train_urban = label_encoder.transform(y_train_urban)
encoded_y_test_urban = label_encoder.transform(y_test_urban)
y_train_categorical_urban = to_categorical(encoded_y_train_urban)
y_test_categorical_urban = to_categorical(encoded_y_test_urban)
label_encoder = LabelEncoder()
label_encoder.fit(y_train_rural)
encoded_y_train_rural = label_encoder.transform(y_train_rural)
encoded_y_test_rural = label_encoder.transform(y_test_rural)
y_train_categorical_rural = to_categorical(encoded_y_train_rural)
y_test_categorical_rural = to_categorical(encoded_y_test_rural)

In [35]:
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=19))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))

model.add(Dense(units=4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network DenseUrban- Loss: {model_loss}, Accuracy DenseUrban: {model_accuracy}")

Epoch 1/60
 - 1s - loss: 1.1606 - accuracy: 0.4057
Epoch 2/60
 - 0s - loss: 0.9770 - accuracy: 0.4713
Epoch 3/60
 - 0s - loss: 0.9502 - accuracy: 0.4426
Epoch 4/60


C:\Programas\Anaconda3\envs\PythonData\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 0s - loss: 0.9306 - accuracy: 0.5041
Epoch 5/60
 - 0s - loss: 0.9225 - accuracy: 0.5287
Epoch 6/60
 - 0s - loss: 0.9083 - accuracy: 0.5738
Epoch 7/60
 - 0s - loss: 0.9017 - accuracy: 0.5287
Epoch 8/60
 - 0s - loss: 0.8719 - accuracy: 0.5738
Epoch 9/60
 - 0s - loss: 0.8704 - accuracy: 0.5984
Epoch 10/60
 - 0s - loss: 0.8974 - accuracy: 0.5410
Epoch 11/60
 - 0s - loss: 0.8530 - accuracy: 0.5902
Epoch 12/60
 - 0s - loss: 0.8178 - accuracy: 0.6516
Epoch 13/60
 - 0s - loss: 0.8505 - accuracy: 0.5779
Epoch 14/60
 - 0s - loss: 0.7878 - accuracy: 0.6598
Epoch 15/60
 - 0s - loss: 0.7782 - accuracy: 0.6352
Epoch 16/60
 - 0s - loss: 0.7815 - accuracy: 0.6475
Epoch 17/60
 - 0s - loss: 0.7531 - accuracy: 0.6516
Epoch 18/60
 - 0s - loss: 0.7661 - accuracy: 0.6475
Epoch 19/60
 - 0s - loss: 0.7680 - accuracy: 0.6557
Epoch 20/60
 - 0s - loss: 0.8324 - accuracy: 0.6516
Epoch 21/60
 - 0s - loss: 0.7120 - accuracy: 0.6803
Epoch 22/60
 - 0s - loss: 0.7080 - accuracy: 0.7049
Epoch 23/60
 - 0s - loss: 0.6

In [ ]:
Normal Neural Network DenseUrban- Loss: 0.7581228744692918, Accuracy DenseUrban: 0.7682926654815674

In [39]:
y_test[:5]

16009    10.0
6975     10.0
11296    10.0
17975     5.0
28153     5.0
Name: tilt, dtype: float64

In [64]:
encoded_predictions = model.predict_classes(X_test_scaled[:100])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels

array([ 5.,  5., 10.,  5., 10.,  5., 10.,  5.,  5.,  5.,  5.,  5.,  5.,
        5., 10.,  5., 10.,  5., 10., 10., 10.,  5., 10.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5., 10., 10., 10.,
       10.,  5., 10., 10., 10.,  5.,  5., 10., 10.,  5., 10., 10.,  5.,
        5., 10.,  5.,  5., 10., 10., 10.,  5.,  5.,  5., 10.,  5.,  5.,
        5.,  5.,  5., 10., 10.,  5., 10.,  5.,  5.,  5., 10.,  5.,  5.,
        5.,  5.,  5.,  5.])

In [41]:
# save fitted model to file
import joblib
filename = 'deep_learning_denseurban_v2.sav'
joblib.dump(model, filename)

['deep_learning_denseurban_v2.sav']

In [ ]:
###SUBURBAN

In [45]:
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=19))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled_urban,
    y_train_categorical_urban,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)
model_loss, model_accuracy = model.evaluate(
    X_test_scaled_urban, y_test_categorical_urban, verbose=2)
print(
    f"Normal Neural Network Suburban- Loss: {model_loss}, Accuracy Suburban: {model_accuracy}")

Epoch 1/60
 - 1s - loss: 0.9835 - accuracy: 0.5280
Epoch 2/60


C:\Programas\Anaconda3\envs\PythonData\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 1s - loss: 0.9202 - accuracy: 0.5789
Epoch 3/60
 - 1s - loss: 0.8947 - accuracy: 0.5879
Epoch 4/60
 - 1s - loss: 0.8798 - accuracy: 0.6008
Epoch 5/60
 - 1s - loss: 0.8607 - accuracy: 0.6147
Epoch 6/60
 - 1s - loss: 0.8577 - accuracy: 0.6050
Epoch 7/60
 - 1s - loss: 0.8378 - accuracy: 0.6153
Epoch 8/60
 - 1s - loss: 0.8433 - accuracy: 0.6130
Epoch 9/60
 - 1s - loss: 0.8267 - accuracy: 0.6241
Epoch 10/60
 - 1s - loss: 0.8246 - accuracy: 0.6170
Epoch 11/60
 - 1s - loss: 0.8099 - accuracy: 0.6266
Epoch 12/60
 - 1s - loss: 0.8086 - accuracy: 0.6301
Epoch 13/60
 - 1s - loss: 0.8005 - accuracy: 0.6317
Epoch 14/60
 - 1s - loss: 0.7959 - accuracy: 0.6370
Epoch 15/60
 - 1s - loss: 0.7919 - accuracy: 0.6393
Epoch 16/60
 - 1s - loss: 0.7784 - accuracy: 0.6501
Epoch 17/60
 - 1s - loss: 0.7696 - accuracy: 0.6513
Epoch 18/60
 - 1s - loss: 0.7595 - accuracy: 0.6617
Epoch 19/60
 - 1s - loss: 0.7622 - accuracy: 0.6589
Epoch 20/60
 - 1s - loss: 0.7498 - accuracy: 0.6658
Epoch 21/60
 - 1s - loss: 0.739

In [60]:
y_test_urban[:60]

30076    10.0
6051     10.0
21065    10.0
13870     5.0
10613    10.0
19161    10.0
6680      5.0
19339     5.0
9384     10.0
14525     5.0
775       5.0
14097    10.0
29189    10.0
14383    10.0
17294     0.0
1597      5.0
17777    10.0
523       5.0
21578    10.0
20989    10.0
5901      0.0
7315      5.0
18532     5.0
4078     10.0
12643     0.0
14422     5.0
20914    10.0
5370      5.0
12807    15.0
5008      5.0
20523     5.0
21986    10.0
4165     10.0
9869     10.0
22892     5.0
4598     10.0
8063     10.0
18039     5.0
15585    10.0
19405     5.0
23638     5.0
10834     5.0
7398      5.0
19514    10.0
21598     5.0
21500    10.0
12549     5.0
14449    10.0
27178     5.0
14619     0.0
14660    10.0
11461    10.0
15209    10.0
5955      5.0
18879     0.0
5712      5.0
9254     10.0
15639     5.0
3091      5.0
10118     5.0
Name: tilt, dtype: float64

In [67]:
encoded_predictions = model.predict_classes(X_test_scaled_urban[:500])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels

array([10., 10., 10., 10.,  5.,  0., 10.,  5., 10.,  5., 10., 10.,  5.,
        5.,  5., 10.,  5., 10.,  0.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5., 10.,  5., 10.,  5., 10.,  5.,
       10.,  5.,  5., 10., 10.,  5., 10.,  5.,  5.,  5.,  5., 10., 10.,
        0.,  5.,  5.,  5.,  5.,  5.,  5.,  5., 10.,  5.,  5., 10., 10.,
       10.,  5.,  0., 10.,  5.,  5., 10., 10.,  5.,  5.,  5.,  5., 10.,
        5.,  5.,  5., 10., 10., 10., 10.,  5.,  5.,  5.,  5.,  5.,  5.,
       10.,  5.,  0.,  5.,  5., 10., 10.,  5.,  5.,  5., 10., 10., 10.,
       10.,  5.,  5., 10., 10.,  5.,  5.,  5., 10., 10., 10.,  0.,  5.,
        5., 10., 10.,  5., 15.,  5.,  5.,  5.,  0., 10.,  5., 10., 10.,
        5., 10.,  5., 10., 10.,  5.,  5., 10.,  5., 10., 10.,  5., 10.,
        5.,  5., 10., 10., 10., 10.,  5., 10.,  5., 10.,  5., 10.,  5.,
        5., 10.,  5.,  5.,  5., 10., 10.,  5.,  5., 10.,  5.,  5.,  5.,
        5.,  5., 10.,  5.,  5., 10.,  5.,  5.,  0.,  5., 10.,  5

In [62]:
X_test_urban[:1]

,cs_call_completion,ps_call_completion,throughput_kbps,rtwp,traffic,quality_ecno,goodquality,users_data,users_total,coverage_rscp,banda,cpich,maxpower,traffic_load,tp_85%,height,mechtilt,hbwidth,vbwidth
30076,99.7084,99.1239,295.4763,-98.7443,148.9967,-11.9004,81.3102,18.8427,38.5793,-83.4629,4437.0,356.0,460.0,89.0,2494.78,42.0,4.0,37.0,9.7


In [52]:
# save fitted model to file
import joblib
filename = 'deep_learning_urban_v2.sav'
joblib.dump(model, filename)

['deep_learning_urban_v2.sav']

In [ ]:
##RURAL

In [53]:
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=19))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled_rural,
    y_train_categorical_rural,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)
model_loss, model_accuracy = model.evaluate(
    X_test_scaled_rural, y_test_categorical_rural, verbose=2)
print(
    f"Normal Neural Network Rural - Loss: {model_loss}, Accuracy Rural: {model_accuracy}")

Epoch 1/60
 - 3s - loss: 1.0243 - accuracy: 0.5317
Epoch 2/60


C:\Programas\Anaconda3\envs\PythonData\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 2s - loss: 0.9830 - accuracy: 0.5543
Epoch 3/60
 - 2s - loss: 0.9714 - accuracy: 0.5590
Epoch 4/60
 - 2s - loss: 0.9588 - accuracy: 0.5654
Epoch 5/60
 - 2s - loss: 0.9518 - accuracy: 0.5663
Epoch 6/60
 - 2s - loss: 0.9471 - accuracy: 0.5696
Epoch 7/60
 - 2s - loss: 0.9348 - accuracy: 0.5741
Epoch 8/60
 - 2s - loss: 0.9299 - accuracy: 0.5763
Epoch 9/60
 - 3s - loss: 0.9225 - accuracy: 0.5835
Epoch 10/60
 - 2s - loss: 0.9140 - accuracy: 0.5818
Epoch 11/60
 - 2s - loss: 0.9098 - accuracy: 0.5839
Epoch 12/60
 - 2s - loss: 0.9001 - accuracy: 0.5928
Epoch 13/60
 - 3s - loss: 0.8897 - accuracy: 0.6005
Epoch 14/60
 - 3s - loss: 0.8859 - accuracy: 0.5982
Epoch 15/60
 - 2s - loss: 0.8749 - accuracy: 0.6003
Epoch 16/60
 - 2s - loss: 0.8670 - accuracy: 0.6062
Epoch 17/60
 - 3s - loss: 0.8634 - accuracy: 0.6107
Epoch 18/60
 - 2s - loss: 0.8552 - accuracy: 0.6141
Epoch 19/60
 - 3s - loss: 0.8459 - accuracy: 0.6157
Epoch 20/60
 - 3s - loss: 0.8299 - accuracy: 0.6216
Epoch 21/60
 - 3s - loss: 0.818

In [54]:
y_test_rural[:5]

6977      0.0
28071     5.0
25088     5.0
16737    10.0
19093     5.0
Name: tilt, dtype: float64

In [70]:
encoded_predictions = model.predict_classes(X_test_scaled_rural[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels

array([ 0.,  5.,  5., 10.,  5.])

In [69]:
X_test_rural[:1]

,cs_call_completion,ps_call_completion,throughput_kbps,rtwp,traffic,quality_ecno,goodquality,users_data,users_total,coverage_rscp,banda,cpich,maxpower,traffic_load,tp_85%,height,mechtilt,hbwidth,vbwidth
6977,99.3917,98.3178,3643.3276,-91.2611,24.096,-5.2664,96.6217,5.7675,8.5123,-83.6823,4413.0,330.0,430.0,47.0,6304.91,40.0,4.0,65.0,7.0


In [56]:
# save fitted model to file
import joblib
filename = 'deep_learning_rural_v2.sav'
joblib.dump(model, filename)

['deep_learning_rural_v2.sav']

In [98]:
testdata=[99.0, 98.5, 1000,
    -95.0, 15.0, -9.0, 85.0, 30.0, 60.0,
    -85.0, 4413, 330, 430, 40.0,
    700, 24.0, 4, 60, 7.5];
    testdata=[testdata]
testdata

[[99.0,
  98.5,
  1000,
  -95.0,
  15.0,
  -9.0,
  85.0,
  30.0,
  60.0,
  -85.0,
  4413,
  330,
  430,
  40.0,
  700,
  24.0,
  4,
  60,
  7.5]]

In [100]:

testdata_input = pd.DataFrame(testdata, columns=["cs_call_completion", "ps_call_completion", "throughput_kbps", "rtwp",
                                           "traffic","quality_ecno","goodquality","users_data","users_total",
                                           "coverage_rscp","banda","cpich","maxpower","traffic_load","tp_85%",
                                           "height","mechtilt","hbwidth","vbwidth"])
testdata_input

,cs_call_completion,ps_call_completion,throughput_kbps,rtwp,traffic,quality_ecno,goodquality,users_data,users_total,coverage_rscp,banda,cpich,maxpower,traffic_load,tp_85%,height,mechtilt,hbwidth,vbwidth
0,99.0,98.5,1000,-95.0,15.0,-9.0,85.0,30.0,60.0,-85.0,4413,330,430,40.0,700,24.0,4,60,7.5


In [101]:
# load and score the student's model
import joblib
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# update file name with student file
filename = 'deep_learning_denseurban.sav'
loaded_model = joblib.load(filename)

label_encoder = LabelEncoder()
label_encoder.fit(y_test)
encoded_y_test = label_encoder.transform(y_test)

y_test_categorical = to_categorical(encoded_y_test)

encoded_predictions = loaded_model.predict_classes(testdata_input)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

# Take number correct over total to get "score" for grading
print(f"Predicted classes: {prediction_labels}")

Predicted classes: [3.]
